Starting your ChipWhisperer (CW) session

In [1]:
import chipwhisperer as cw
import matplotlib.pyplot as plt

In [2]:
scope = cw.scope() #Initializing the CW scope
# scope.upgrade_firmware()

In [3]:
target = cw.target(scope, cw.targets.SimpleSerial) #programming your target

In [4]:
scope.gain.db = 35

In [5]:
scope #gives the scope's info; helpful when debugging if errors are thrown.

cwhusky Device
sn             = 50203220343043543030372037303038
fpga_buildtime = 4/11/2024, 09:41
fw_version = 
    major = 1
    minor = 5
    debug = 0
gain = 
    mode = high
    gain = 44
    db   = 35.18348623853211
adc = 
    state                    = False
    basic_mode               = rising_edge
    timeout                  = 2
    offset                   = 0
    presamples               = 0
    samples                  = 131124
    decimate                 = 1
    trig_count               = 0
    stream_mode              = False
    test_mode                = False
    bits_per_sample          = 12
    segments                 = 1
    segment_cycles           = 0
    segment_cycle_counter_en = False
    clip_errors_disabled     = False
    lo_gain_errors_disabled  = False
    errors                   = gain too low error, 
clock = 
    clkgen_src             = system
    clkgen_freq            = 0
    adc_mul                = 4
    adc_freq               = 0
    freq_ctr 

In [6]:
scope.default_setup()

scope.gain.gain                          changed from 44                        to 22                       
scope.gain.db                            changed from 35.18348623853211         to 25.091743119266056       
scope.adc.samples                        changed from 131124                    to 5000                     
scope.clock.clkgen_freq                  changed from 0                         to 7370129.87012987         
scope.clock.adc_freq                     changed from 0                         to 29480519.48051948        
scope.clock.extclk_monitor_enabled       changed from True                      to False                    
scope.clock.extclk_tolerance             changed from 1144409.1796875           to 13096723.705530167       
scope.io.tio1                            changed from serial_tx                 to serial_rx                
scope.io.tio2                            changed from serial_rx                 to serial_tx                
scope.io.hs2       

Cell below starts the printer and captures traces at a total of 9 positions on the radio chip. It finds the "optimal" position by averaging the numbers. 
Then, it continues to collect more traces (trace_count) at the best position. 

In [9]:
import serial
import time
import numpy as np
import os

# Printer Configuration
SERIAL_PORT = '/dev/ttyUSB0'  # Replace with your serial port
BAUD_RATE = 115200  # Typical baud rate for Ender 3
TIMEOUT = 1  # Timeout for printer communication

# File Configuration
FILE_DIRECTORY = '/home/cyse/chipwhisperer/jupyter/courses/sca101/2025_Final_Data_Collection/Phone/Radio_Station'
TRIALS = 3
TRACE_COUNT = 10  # Number of traces at best position

GCODE_INIT = [
    "G21", "G90", "M140 S0", "M104 S0", "M107", "G28", "G0 Z25.4 F3000", "G0 X22.7 Y116.8 F3000", "G0 Z0.0 F3000"
]

GCODE_POSITIONS = [
    "G1 X22.7 Y116.8 Z0.0 F1500", "G1 X27.78 Y116.8 Z0.0 F1500", "G1 X32.86 Y116.8 Z0.0 F1500", 
    "G1 X32.86 Y121.88 Z0.0 F1500", "G1 X27.78 Y121.88 Z0.0 F1500", "G1 X22.7 Y121.88 Z0.0 F1500", 
    "G1 X22.7 Y126.96 Z0.0 F1500", "G1 X27.78 Y126.96 Z0.0 F1500", "G1 X32.86 Y126.96 Z0.0 F1500"
]

GCODE_FINAL = ["G0 Z25.4 F3000", "M140 S0", "M104 S0", "M84"]

def collect_traces():
    scope.arm()
    print("Chipwhisperer armed")
    scope.capture()
    print("Capture done!")
    power_trace = scope.get_last_trace()
    print(f"Power trace captured: {power_trace}")
    return power_trace

def send_gcode(ser, command):
    ser.write((command + '\n').encode())
    print(f"Sent: {command}")
    time.sleep(6)

def collect_data_at_positions():
    position_averages = np.zeros((TRIALS, len(GCODE_POSITIONS)))
    
    with serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=TIMEOUT) as ser:
        print("Printer connected. Sending initialization G-code.")
        for command in GCODE_INIT:
            send_gcode(ser, command)
        
        for trial in range(1, TRIALS + 1):
            print(f"Starting trial {trial}...")
            trace_list = []
            for idx, position in enumerate(GCODE_POSITIONS):
                print(f"Moving to position {idx + 1}: {position}")
                send_gcode(ser, position)
                trace = collect_traces()
                trace_list.append(trace)
                position_averages[trial - 1, idx] = np.mean(np.abs(trace))
                print(f"Captured trace for position {idx + 1}")
                time.sleep(2)
            
            np.save(os.path.join(FILE_DIRECTORY, f'trial{trial}.npy'), np.array(trace_list))
            print(f"Trial {trial} data saved.")
        
        print("Sending final G-code.")
        for command in GCODE_FINAL:
            send_gcode(ser, command)
    
    return position_averages

def determine_best_position(position_averages):
    final_averages = np.mean(position_averages, axis=0)
    best_idx = np.argmax(final_averages)
    print("Final Averages for Each Position:")
    for i, avg in enumerate(final_averages, start=1):
        print(f"Position {i}: {avg:.4f} ({GCODE_POSITIONS[i - 1]})")
    print(f"\nBest position is {best_idx + 1} with G-code: {GCODE_POSITIONS[best_idx]}")
    return best_idx, GCODE_POSITIONS[best_idx]

def collect_data_at_best_position(best_position):
    trace_list = []
    
    with serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=TIMEOUT) as ser:
        print("Printer connected. Sending initialization G-code.")
        for command in GCODE_INIT:
            send_gcode(ser, command)
        
        print(f"Moving to best position: {best_position}")
        send_gcode(ser, best_position)
        
        print(f"Collecting {TRACE_COUNT} traces at the best position.")
        for idx in range(TRACE_COUNT):
            trace = collect_traces()
            trace_list.append(trace)
            print(f"Captured trace {idx + 1}/{TRACE_COUNT}")
            time.sleep(2)
        
        print("Sending final G-code.")
        for command in GCODE_FINAL:
            send_gcode(ser, command)
    
    np.save(os.path.join(FILE_DIRECTORY, '5G_Traces_Phone_Radio_Station_Test_4.npy'), np.array(trace_list))
    print("Final traces saved.")

def main():
    print("Starting data collection at multiple positions.")
    position_averages = collect_data_at_positions()
    best_idx, best_position = determine_best_position(position_averages)
    print(f"Best position determined: {best_idx + 1} - {best_position}")
    collect_data_at_best_position(best_position)
    print("Data collection complete.")

if __name__ == "__main__":
    main()


Starting data collection at multiple positions.
Printer connected. Sending initialization G-code.
Sent: G21
Sent: G90
Sent: M140 S0
Sent: M104 S0
Sent: M107
Sent: G28
Sent: G0 Z25.4 F3000
Sent: G0 X22.7 Y116.8 F3000
Sent: G0 Z0.0 F3000
Starting trial 1...
Moving to position 1: G1 X22.7 Y116.8 Z0.0 F1500
Sent: G1 X22.7 Y116.8 Z0.0 F1500
Chipwhisperer armed


(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 13
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 12


Capture done!
Power trace captured: [ 0.00415039  0.00341797  0.01611328 ... -0.03076172 -0.03735352
 -0.02929688]
Captured trace for position 1
Moving to position 2: G1 X27.78 Y116.8 Z0.0 F1500
Sent: G1 X27.78 Y116.8 Z0.0 F1500
Chipwhisperer armed


(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 13
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 12


Capture done!
Power trace captured: [0.00366211 0.01391602 0.00756836 ... 0.04077148 0.03588867 0.03271484]
Captured trace for position 2
Moving to position 3: G1 X32.86 Y116.8 Z0.0 F1500
Sent: G1 X32.86 Y116.8 Z0.0 F1500
Chipwhisperer armed


...

(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 13
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 12


Capture done!
Power trace captured: [ 0.02758789  0.0246582   0.01928711 ... -0.01928711 -0.00610352
 -0.01123047]
Captured trace for position 6
Moving to position 7: G1 X22.7 Y126.96 Z0.0 F1500
Sent: G1 X22.7 Y126.96 Z0.0 F1500
Chipwhisperer armed


(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 13
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 12


Capture done!
Power trace captured: [ 0.03173828  0.02490234  0.03320312 ... -0.03295898 -0.02246094
 -0.0246582 ]
Captured trace for position 7
Moving to position 8: G1 X27.78 Y126.96 Z0.0 F1500
Sent: G1 X27.78 Y126.96 Z0.0 F1500


KeyboardInterrupt: 

The cell below is another version of the code that will collect traces at a single position.  That position is defined in the GCODE_INIT. The number of traces to be collected is defined in TRACE_COUNT. 

In [12]:
import serial
import time
import numpy as np
import os

# Printer Configuration
SERIAL_PORT = '/dev/ttyUSB0'  # Replace with your serial port
BAUD_RATE = 115200  # Typical baud rate for Ender 3
TIMEOUT = 1  # Timeout for printer communication

######### File Configuration ###########
FILE_DIRECTORY = '/home/cyse/chipwhisperer/jupyter/courses/sca101/2025_Final_Data_Collection/Phone/Radio_Station'

FILENAME = '5G_Traces_Phone_Radio_Station_Test_7.npy'
########################################

# Complete G-code sequence
GCODE_INIT = [
    "G21",  # Set units to millimeters
    "G90",  # Absolute positioning
    "M140 S0",  # Disable bed heating
    "M104 S0",  # Disable extruder heating
    "M107",  # Fan off
    "G28",  # Home all axes
    "G0 Z25.4 F3000",  # Raise Z-axis 1 inch
    "G0 X22.7 Y116.8 F3000",  # Move to the top-left corner of the grid
    "G0 Z0.0 F3000"  # Lower Z-axis to starting position
]

GCODE_FINAL = [
    "G0 Z25.4 F3000",  # Raise Z-axis 1 inch after finishing the grid
    "M140 S0",  # Turn off bed heating
    "M104 S0",  # Turn off extruder heating
    "M84"  # Disable motors
]

# ChipWhisperer Data Collection
def collect_traces():
    # """Collect power traces using ChipWhisperer."""
    scope.arm()
    print("Chipwhisperer armed")

    scope.capture()
    print("Capture done!")

    power_trace = scope.get_last_trace()
    print(f"Power trace captured: {power_trace}")

    return power_trace

def send_gcode_to_printer(ser, gcode_command):
    # """Send a G-code command to the printer."""
    ser.write((gcode_command + '\n').encode())
    print(f"Sent: {gcode_command}")

def main():
    # """Main routine to control the printer and collect traces."""
    trace_list = []
    TARGET_POSITION = "G1 X27.78 Y121.88 Z0.0 F1500"  # Replace with your desired G-code position
    TRACE_COUNT = 200  # Number of traces to collect

    with serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=TIMEOUT) as ser:
        print("Printer connected. Sending initialization G-code.")

        # Send initialization G-code
        for command in GCODE_INIT:
            send_gcode_to_printer(ser, command)
            time.sleep(2)  # Maintain a 6-second delay for initialization

        # Move to the target position
        print(f"Moving to target position: {TARGET_POSITION}")
        send_gcode_to_printer(ser, TARGET_POSITION)
        time.sleep(2)  # Wait for movement to complete

        # print("Waiting 40 seconds before starting trace collection...")
        # time.sleep(40)

        # Collect 25 traces at the target position
        print(f"Collecting {TRACE_COUNT} traces at the target position.")
        for idx in range(TRACE_COUNT):
            # Arm and capture trace
            trace = collect_traces()
            trace_list.append(trace)
            print(f"Captured trace {idx + 1}/{TRACE_COUNT}")

            # Optional: Wait time between traces
            time.sleep(1)

        # Send final G-code
        print("Sending final G-code.")
        for command in GCODE_FINAL:
            send_gcode_to_printer(ser, command)
            time.sleep(1)  # Maintain a 6-second delay for finalization

    # Save all collected traces to file
    trace_array = np.array(trace_list)
    save_path = os.path.join(FILE_DIRECTORY, FILENAME)
    
    if os.path.exists(save_path):
        existing_data = np.load(save_path, allow_pickle=True)
        all_traces = np.concatenate((existing_data, trace_array))
    else:
        all_traces = trace_array
    
    np.save(save_path, all_traces)
    print(f"Traces saved to {save_path}. Total traces collected: {len(trace_list)}.")


if __name__ == "__main__":
    main()


Printer connected. Sending initialization G-code.
Sent: G21
Sent: G90
Sent: M140 S0
Sent: M104 S0
Sent: M107
Sent: G28
Sent: G0 Z25.4 F3000
Sent: G0 X22.7 Y116.8 F3000
Sent: G0 Z0.0 F3000
Moving to target position: G1 X27.78 Y121.88 Z0.0 F1500
Sent: G1 X27.78 Y121.88 Z0.0 F1500
Chipwhisperer armed


(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 13
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 12


Capture done!
Power trace captured: [ 0.01513672  0.01367188  0.01757812 ... -0.00537109 -0.00537109
 -0.00366211]
Captured trace 1/200
Chipwhisperer armed


(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 13
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 12


Capture done!
Power trace captured: [-0.01269531 -0.01269531 -0.01196289 ...  0.01977539  0.01123047
  0.01489258]
Captured trace 2/200
Chipwhisperer armed


(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 13
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 12


Capture done!
Power trace captured: [-0.02001953 -0.02075195 -0.01782227 ...  0.00317383  0.01831055
  0.00561523]
Captured trace 3/200
Chipwhisperer armed


KeyboardInterrupt: 

The cell below is to disable the current chipwhisperer scope and target.

In [142]:
scope.dis()
target.dis()